In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, ImageDataGenerator
from tensorflow.keras.utils import to_categorical

load data

In [2]:
def images_to_sequence(folder, target_size=(224, 224), sequence_length=224):
    sequences_list = []
    labels = []
    for label in ["HC", "PD"]:
        label_path = os.path.join(folder, label)
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            img = load_img(image_path, color_mode="grayscale", target_size=target_size)
            image_array = img_to_array(img)
            img_array = image_array / 255.0 #normalize
            sequences = img_array.reshape((sequence_length, -1))
            sequences_list.append(sequences)
            labels.append(label)
            
    sequences = np.array(sequences_list)
    labels = np.array(labels)
    return sequences, labels

drawings_sequences, drawings_labels = images_to_sequence("drawings")
speech_sequences, speech_labels = images_to_sequence("speech")

#encode labels to numerical
label_encoder = LabelEncoder()
drawings_labels_encoded = label_encoder.fit_transform(drawings_labels)
speech_labels_encoded = label_encoder.fit_transform(speech_labels)

data augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [7]:
def create_rnn_model(input_shape, num_units=128):
    model = Sequential()
    model.add(LSTM(num_units, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(num_units))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    
    return model

# RNN models
drawings_rnn = create_rnn_model(input_shape=drawings_sequences[0].shape)
speech_rnn = create_rnn_model(input_shape=speech_sequences[0].shape)

data processing

In [9]:
drawings_label_oh= to_categorical(drawings_labels_encoded)
drawings_rnn.fit(drawings_sequences, drawings_label_oh, epochs=10, batch_size=32, validation_split=0.2)

speech_label_oh = to_categorical(speech_labels_encoded)
speech_rnn.fit(speech_sequences, speech_label_oh, epochs=10, batch_size=32, validation_split=0.2)

SyntaxError: positional argument follows keyword argument (2295855561.py, line 5)

train model

evaluate, ensemble

In [5]:
drawings_rnn.save('drawings_model.h5')
speech_rnn.save('speech_model.h5')


c:\Users\datng\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
